In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('./data/historical_sales.csv')

In [5]:
past_sales = data[data['SALE TYPE'] == 'PAST SALE']

In [12]:
past_sales = past_sales[['SALE TYPE' , 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE OR PROVINCE', 'PRICE',
               'BEDS', 'BATHS','LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', '$/SQUARE FEET', 'HOA/MONTH',
           'MLS#', 'LATITUDE','LONGITUDE']]

In [49]:
past_sales = past_sales[past_sales['LOT SIZE']< 3000000]

In [50]:
past_sales = past_sales.dropna(how='any',axis=0) 


In [51]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_sales = past_sales.select_dtypes(include=numerics)

In [55]:
kmeans = KMeans(n_clusters=7, random_state=0).fit(numeric_sales)


In [56]:
past_sales['cluster'] = kmeans.labels_

In [80]:
past_sales.groupby('cluster').count()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,MLS#,LATITUDE,LONGITUDE
cluster,,,,,,,,,,,,,,,,,,
0,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946,17946
1,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955,4955
2,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614,20614
3,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111,111
4,554,554,554,554,554,554,554,554,554,554,554,554,554,554,554,554,554,554
5,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285,1285
6,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152,12152


In [86]:
past_sales= past_sales[past_sales['cluster'] == 6]

In [87]:
for_linear = past_sales[['BEDS','BATHS','SQUARE FEET','LOT SIZE', 'YEAR BUILT',
                         '$/SQUARE FEET', 'HOA/MONTH', 'cluster']]
                        

In [88]:
target = past_sales['PRICE']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(for_linear, target, test_size = 0.2)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [85]:
lm = LinearRegression()
lm.fit(X_train, y_train)
lm.score(X_train, y_train)
lm.score(X_test, y_test)

-2.2896595038964382